# 1. TELEPORTATION
Name: Sai Krishna Teja Varma Manthena\
UnityId: smanthe

In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import IBMQ, Aer, transpile, assemble, execute

In [2]:
#function to convert string to a list of bits
def stringToBitList(s):
     ords = (ord(c) for c in s)
     shifts = (7, 6, 5, 4, 3, 2, 1, 0)
     return [(o >> shift) & 1 for o in ords for shift in shifts]
 
#function to convert list of bits to subsequent characters
def bitlistToCharacters(bl):
     bi = iter(bl)
     bytes = zip(*(bi,) * 8)
     shifts = (7, 6, 5, 4, 3, 2, 1, 0)
     for byte in bytes:
         yield chr(sum(bit << s for bit, s in zip(byte, shifts)))
 
#function to convert list of bits to the string
def bitlistToString(bl):
     return ''.join(bitlistToCharacters(bl))

In [3]:
def bellPair(circ, a, b):
    circ.h(a) 
    circ.cx(a,b) 

def gatesAlice(circ, a, b):
    circ.cx(a, b)
    circ.h(a)

def gatesBob(circ, qubit, crz, crx):
    circ.x(qubit).c_if(crx, 1)  
    circ.z(qubit).c_if(crz, 1) 

In [4]:
def compute_teleportation(b):
    sim = Aer.get_backend('qasm_simulator')
    sol = []
    for i in b:
        q = QuantumRegister(3, name="q")
        crz, crx = ClassicalRegister(1, name="crz"), ClassicalRegister(1, name="crx")
        teleportation_circ = QuantumCircuit(q, crz, crx)
        cr_result = ClassicalRegister(1)
        teleportation_circ.add_register(cr_result)
        
        if i == 1:
            teleportation_circ.x(0)
        bellPair(teleportation_circ, 1, 2)

        teleportation_circ.barrier()
        gatesAlice(teleportation_circ, 0, 1)

        teleportation_circ.barrier()
        teleportation_circ.measure(0,0)
        teleportation_circ.measure(1,1)


        teleportation_circ.barrier()
        gatesBob(teleportation_circ, 2, crz, crx)
        
        teleportation_circ.measure(2, 2)
        
        result = execute(teleportation_circ, backend=sim, shots=1).result()
        counts = result.get_counts()
        
        sol.append(int(list(counts.keys())[0][0]))
    return sol

In [5]:
s = input("Input by Alice (string):\n")
b = stringToBitList(s)
sol = compute_teleportation(b)
print(bitlistToString(sol))

Input by Alice (string):
Hello, quantum
Hello, quantum
